In [1]:
# pip install -r requirements.txt

In [1]:
import cv2
import numpy as np

In [17]:
DIGITS_LOOKUP = {
	(1, 1, 1, 0, 1, 1, 1): 0,
	(0, 0, 1, 0, 0, 1, 0): 1,
	(1, 0, 1, 1, 1, 0, 1): 2,
	(1, 0, 1, 1, 0, 1, 1): 3,
	(0, 1, 1, 1, 0, 1, 0): 4,
	(1, 1, 0, 1, 0, 1, 1): 5,
	(1, 1, 0, 1, 1, 1, 1): 6,
	(1, 0, 1, 0, 0, 1, 0): 7,
	(1, 1, 1, 1, 1, 1, 1): 8,
	(1, 1, 1, 1, 0, 1, 1): 9,
    (1, 1, 1, 1, 1, 1, 0): 'A',
    (1, 1, 0, 1, 1, 0, 1): 'E',
    (1, 1, 0, 1, 1, 0, 0): 'F',
    (0, 1, 1, 1, 1, 1, 0): 'H',
    (0, 1, 0, 0, 1, 0, 1): 'L'
}

In [3]:
def getDigits(originalimg, thresholdimg, contours):

    start_index=0
    cropped_binary_matrix = []
    digits = []
    for contour in contours:
            # Find the bounding rectangle for each contour
            x, y, w, h = cv2.boundingRect(contour)
            if(w > 5 and (h > w*1.2)):
                
                start_index += 1

                print(x, y, w, h)

                # Crop the image to the bounding rectangle
                cropped_image = []
                cropped_image = thresholdimg[y:y+h, x:x+w]
                cropped_binary_matrix.append(cropped_image)

                #cv2.rectangle(image_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
                #save the image
                #cv2.imwrite(f'image{start_index}.png', cropped_image)
                cv2.imshow('7segment', cropped_binary_matrix[start_index-1])
                cv2.waitKey(1000)

                (roiH, roiW) = cropped_image.shape
                (dW, dH) = (int(roiW * 0.4), int(roiH * 0.15))
                dHC = int(roiH * 0.1)
                
                segments = [
                    ((0, 0), (w, dH)),	# top
                    ((int(dW*0.2), dH), (dW, h // 2)),	# top-left
                    ((w - dW + int(dW*0.3), dH), (w, h // 2)),	# top-right
                    ((0, (h // 2) - dHC) , (w, (h // 2) + dHC)), # center
                    ((0, h // 2), (dW, h - dH)),	# bottom-left
                    ((w - dW, h // 2), (w, h - dH)),	# bottom-right
                    ((0, h - dH), (w, h))	# bottom
                ]

                """Original
                segments = [
                    ((0, 0), (w, dH)),	# top
                    ((0, 0), (dW, h // 2)),	# top-left
                    ((w - dW, 0), (w, h // 2)),	# top-right
                    ((0, (h // 2) - dHC) , (w, (h // 2) + dHC)), # center
                    ((0, h // 2), (dW, h)),	# bottom-left
                    ((w - dW, h // 2), (w, h)),	# bottom-right
                    ((0, h - dH), (w, h))	# bottom
                ]
                """
                on = [0] * len(segments)

                #cv2.rectangle(image, (0, 0), (w, dH), (0, 255, 0), 1)



                if(w > h*0.4):
                    for (i, ((xA, yA), (xB, yB))) in enumerate(segments):
                        segROI = cropped_image[yA:yB, xA:xB]
                        
                        total = cv2.countNonZero(segROI)
                        area = (xB - xA) * (yB - yA)
                        print("total", total, "area", area)
                        if total / float(area) > 0.4:
                            on[i] = 1
                        else:
                            on[i] = 0
                        print(on[i])
        
                        cv2.imshow('Segments', segROI)
                        cv2.waitKey(0)
                        cv2.destroyWindow('Segments')
                else:
                    on = [0, 0, 1, 0, 0, 1, 0]

                digit = DIGITS_LOOKUP[tuple(on)]
                digits.append(digit)
                cv2.rectangle(originalimg, (x, y), (x + w, y + h), (0, 255, 0), 1)
                cv2.putText(originalimg, str(digit), (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)
                cv2.destroyWindow('7segment')

    cv2.destroyAllWindows()
    return digits

In [30]:
videostream = 1


cap = cv2.VideoCapture()
cap.open("http://192.168.1.62:8080/?action=stream")
ret, image = cap.read()
if(videostream==0):
    image = cv2.imread("image1.png")
    #image = cv2.imread("challange/1.png")


gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

_, threshold = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)


kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (2, 2))
thresh = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel)

cv2.imshow('Anzeige1', gray)
#cv2.imshow('Anzeige2', None)
cv2.imshow('Anzeige3', threshold)
cv2.imshow('Anzeige4', thresh)

cv2.waitKey(0)
 
cv2.destroyAllWindows()


In [31]:
print(len(contours))

3


In [32]:
if (len(contours)<5):
    digits = getDigits(image, thresh, contours)
    print(digits)

187 171 31 39
total 95 area 155
1
total 82 area 140
1
total 82 area 126
1
total 55 area 186
0
total 91 area 180
1
total 89 area 180
1
total 121 area 155
1
323 163 27 39
total 78 area 135
1
total 19 area 112
0
total 79 area 98
1
total 101 area 162
1
total 42 area 150
0
total 92 area 150
1
total 97 area 135
1
356 160 32 41
total 90 area 192
1
total 82 area 140
1
total 86 area 126
1
total 81 area 256
0
total 78 area 180
1
total 94 area 180
1
total 133 area 192
1
[0, 3, 0]


In [33]:
cv2.imshow('Anzeige1', gray)
cv2.imshow('Anzeige3', threshold)
cv2.imshow('Anzeige4', thresh)
cv2.imshow("output", image)
cv2.waitKey(0)

cap.release() 
cv2.destroyAllWindows()